In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
from torch_geometric.data import Data 
from sklearn.decomposition import PCA
torch.__version__

'2.1.1'

In [3]:
import torch

# PyTorch 버전 확인
print("PyTorch 버전:", torch.__version__)

# CUDA 사용 가능 여부 및 버전 확인
if torch.cuda.is_available():
    print("CUDA 사용 가능")
    print("CUDA 버전:", torch.version.cuda)
else:
    print("CUDA 사용 불가")

PyTorch 버전: 2.1.1
CUDA 사용 가능
CUDA 버전: 12.1


In [4]:
df=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/df.csv',encoding='cp949')

In [5]:
df['상차시간'] = pd.to_datetime(df['상차시간'])
df['하차시간'] = pd.to_datetime(df['하차시간'])
df['등록시간'] = pd.to_datetime(df['등록시간'])
df['배차시간'] = pd.to_datetime(df['배차시간'])
df['등록날짜'] = pd.to_datetime(df['등록날짜'])
df['운행시간'] = pd.to_timedelta(df['운행시간'])
df['잡힌시간'] = pd.to_timedelta(df['잡힌시간'])
df['상차시간'] = df['상차시간'].astype(np.int64) // 10**9
df['하차시간'] = df['하차시간'].astype(np.int64) // 10**9
df['운행시간'] = df['운행시간'].astype(np.int64) // 10**9
df['잡힌시간'] = df['잡힌시간'].astype(np.int64) // 10**9
df['등록시간'] = df['등록시간'].astype(np.int64) // 10**9
df['배차시간'] = df['배차시간'].astype(np.int64) // 10**9

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903198 entries, 0 to 903197
Data columns (total 27 columns):
 #   Column  Non-Null Count   Dtype              
---  ------  --------------   -----              
 0   화물키     903198 non-null  int64              
 1   화물정보    903198 non-null  object             
 2   상차지     903198 non-null  object             
 3   상차경도    903198 non-null  float64            
 4   상차위도    903196 non-null  float64            
 5   하차지     903198 non-null  object             
 6   하차경도    903198 non-null  float64            
 7   하차위도    903198 non-null  float64            
 8   차종류     903198 non-null  object             
 9   차톤수     903198 non-null  object             
 10  상차시간    903198 non-null  int64              
 11  하차시간    903198 non-null  int64              
 12  결제방법    903198 non-null  object             
 13  기사운임    903198 non-null  int64              
 14  화물종류    903198 non-null  object             
 15  상차방법    903198 non-null  object   

In [7]:
# 인덱스를 기준으로 데이터 프레임을 분할
train_df = df[df.index <= 758353] # 4월 ~ 8월
test_df = df[df.index > 758353] # 9월

# 훈련 세트와 테스트 세트의 크기 확인
print("훈련 세트 크기:", train_df.shape)
print("테스트 세트 크기:", test_df.shape)

# 84% 대 16%

훈련 세트 크기: (758354, 27)
테스트 세트 크기: (144844, 27)


In [8]:
# 차주 특성 선택
driver_features = train_df[['차주키', '차종류', '차톤수', '기사운임', '결제방법']].set_index('차주키')
driver_features_test = test_df[['차주키', '차종류', '차톤수', '기사운임', '결제방법']].set_index('차주키')

In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# 정규화를 위한 함수 정의
def apply_normalization(df, numeric_columns):
    scaler = MinMaxScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    return df, scaler

# 원-핫 인코딩을 위한 함수 정의 (OneHotEncoder 사용)
def apply_one_hot_encoding(df, categorical_columns):
    encoder = OneHotEncoder(sparse=False)
    encoded_data = encoder.fit_transform(df[categorical_columns])
    encoded_df = pd.DataFrame(encoded_data, index=df.index, columns=encoder.get_feature_names_out(categorical_columns))
    return pd.concat([df.drop(categorical_columns, axis=1), encoded_df], axis=1), encoder

# 범주형 특성과 수치형 특성 선택
categorical_columns = ['차종류', '차톤수', '결제방법']
numeric_columns = ['기사운임']

# 먼저, 수치형 특성에 대해 정규화를 적용합니다.
driver_features, scaler = apply_normalization(driver_features, numeric_columns)
driver_features_test[numeric_columns] = scaler.transform(driver_features_test[numeric_columns])

# 이후, 범주형 특성에 대해 원-핫 인코딩을 적용합니다.
driver_features, encoder = apply_one_hot_encoding(driver_features, categorical_columns)
driver_features_test, _ = apply_one_hot_encoding(driver_features_test, categorical_columns)


C:\Users\user\anaconda3\envs\ys\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\envs\ys\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
missing_columns = ['결제방법_후불','결제방법_일반']

for column in missing_columns:
    driver_features[column] = 0

# 확인을 위해 수정된 데이터프레임의 첫 5행 출력
print(driver_features.head())

          기사운임  차종류_냉탑  차종류_다마스  차종류_라보  차종류_윙바디  차종류_윙축  차종류_추레라  차종류_카고  \
차주키                                                                         
47982  0.00180     0.0      0.0     0.0      1.0     0.0      0.0     0.0   
85445  0.00162     0.0      0.0     0.0      1.0     0.0      0.0     0.0   
83154  0.00270     0.0      0.0     0.0      1.0     0.0      0.0     0.0   
54972  0.00234     0.0      0.0     0.0      0.0     0.0      0.0     0.0   
51336  0.00315     0.0      0.0     0.0      0.0     0.0      0.0     0.0   

       차종류_카고윙바디  차종류_카고축  ...  차톤수_8톤  차톤수_9.5톤  차톤수_기타  결제방법_별도협의  결제방법_선불  \
차주키                        ...                                                 
47982        0.0      0.0  ...     0.0       0.0     0.0        0.0      0.0   
85445        0.0      0.0  ...     0.0       0.0     0.0        0.0      0.0   
83154        0.0      0.0  ...     0.0       0.0     0.0        0.0      0.0   
54972        1.0      0.0  ...     0.0       0.0     0.0    

In [11]:
missing_columns = ['차종류_다마스']

for column in missing_columns:
    driver_features_test[column] = 0

# 확인을 위해 수정된 데이터프레임의 첫 5행 출력
print(driver_features_test.head())

          기사운임  차종류_냉탑  차종류_라보  차종류_윙바디  차종류_윙축  차종류_추레라  차종류_카고  차종류_카고윙바디  \
차주키                                                                           
56129  0.00315     0.0     0.0      1.0     0.0      0.0     0.0        0.0   
85477  0.00261     0.0     0.0      0.0     0.0      0.0     0.0        1.0   
22569  0.00450     0.0     0.0      1.0     0.0      0.0     0.0        0.0   
84744  0.00315     0.0     0.0      1.0     0.0      0.0     0.0        0.0   
21957  0.00252     0.0     0.0      1.0     0.0      0.0     0.0        0.0   

       차종류_카고축  차종류_탑  ...  차톤수_9.5톤  차톤수_기타  결제방법_별도협의  결제방법_선불  결제방법_인수증  \
차주키                    ...                                                   
56129      0.0    0.0  ...       0.0     0.0        0.0      0.0       1.0   
85477      0.0    0.0  ...       0.0     0.0        0.0      0.0       1.0   
22569      0.0    0.0  ...       0.0     0.0        0.0      0.0       1.0   
84744      0.0    0.0  ...       0.0     0.0        0.0 

In [12]:
# 화물 특성 선택
cargo_features = train_df[['화물키','차주키', '화물종류','화물실중량','카테고리','상차방법', '하차방법']].set_index('화물키') 
cargo_features_test = test_df[['화물키','차주키','화물종류','화물실중량','카테고리','상차방법', '하차방법']].set_index('화물키')

In [13]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# 정규화를 위한 함수 정의
def apply_normalization(df, numeric_columns):
    scaler = MinMaxScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    return df, scaler

# 원-핫 인코딩을 위한 함수 정의 (OneHotEncoder 사용)
def apply_one_hot_encoding(df, categorical_columns):
    encoder = OneHotEncoder(sparse=False)
    encoded_data = encoder.fit_transform(df[categorical_columns])
    encoded_df = pd.DataFrame(encoded_data, index=df.index, columns=encoder.get_feature_names_out(categorical_columns))
    return pd.concat([df.drop(categorical_columns, axis=1), encoded_df], axis=1), encoder

# 범주형 특성과 수치형 특성 선택
cargo_categorical_columns = ['화물종류', '카테고리', '상차방법', '하차방법']
cargo_numeric_columns = ['화물실중량']

# 먼저, 수치형 특성에 대해 정규화를 적용합니다.
cargo_features, cargo_scaler = apply_normalization(cargo_features, cargo_numeric_columns)
cargo_features_test[cargo_numeric_columns] = cargo_scaler.transform(cargo_features_test[cargo_numeric_columns])

# 이후, 범주형 특성에 대해 원-핫 인코딩을 적용합니다.
cargo_features, cargo_encoder = apply_one_hot_encoding(cargo_features, cargo_categorical_columns)
cargo_features_test, _ = apply_one_hot_encoding(cargo_features_test, cargo_categorical_columns)

C:\Users\user\anaconda3\envs\ys\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\envs\ys\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [14]:
missing_columns = ['화물종류_고체산적', '화물종류_액체산적', '화물종류_컨테이너']

for column in missing_columns:
    cargo_features_test[column] = 0

# 확인을 위해 수정된 데이터프레임의 첫 5행 출력
print(cargo_features_test.head())

          차주키     화물실중량  화물종류_일반카고  카테고리_가구  카테고리_금속  카테고리_기계  카테고리_기타  \
화물키                                                                      
102114  56129  0.407407        1.0      0.0      0.0      0.0      0.0   
121351  85477  0.333333        1.0      0.0      0.0      0.0      0.0   
118689  22569  0.259259        1.0      0.0      0.0      0.0      0.0   
118590  84744  0.296296        1.0      0.0      0.0      0.0      0.0   
118545  21957  0.407407        1.0      0.0      0.0      0.0      0.0   

        카테고리_날짜  카테고리_단열재  카테고리_드럼  ...  상차방법_호이스트  하차방법_수작업  하차방법_일반  \
화물키                                 ...                                 
102114      0.0       0.0      0.0  ...        0.0       0.0      1.0   
121351      0.0       0.0      0.0  ...        0.0       0.0      1.0   
118689      0.0       0.0      0.0  ...        0.0       0.0      1.0   
118590      1.0       0.0      0.0  ...        0.0       0.0      1.0   
118545      0.0       0.0      0.0  ...    

In [15]:
# 엣지 특성 선택
edge_features = train_df[['차주키','화물키','상차경도', '상차위도', '하차경도', '하차위도', '상차시간', '하차시간', '직선거리', '등록시간','배차시간','요일','운행시간','잡힌시간']]
edge_features_test = test_df[['차주키','화물키','상차경도', '상차위도', '하차경도', '하차위도', '상차시간', '하차시간', '직선거리', '등록시간','배차시간','요일','운행시간','잡힌시간']]

In [16]:
edge_features.fillna(0, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_17268\2812238908.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edge_features.fillna(0, inplace=True)


In [17]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import pandas as pd

# 원-핫 인코딩 및 정규화를 위한 함수 정의
def apply_one_hot_encoding(df, categorical_columns):
    encoder = OneHotEncoder(sparse=False)
    encoded_data = encoder.fit_transform(df[categorical_columns])
    encoded_df = pd.DataFrame(encoded_data, index=df.index, columns=encoder.get_feature_names_out(categorical_columns))
    return pd.concat([df.drop(categorical_columns, axis=1), encoded_df], axis=1), encoder

def apply_normalization(df, numeric_columns):
    scaler = MinMaxScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    return df, scaler

# 범주형 및 수치형 특성 선택
edge_categorical_columns = ['요일']
edge_numeric_columns = ['상차경도', '상차위도', '하차경도', '하차위도', '직선거리','등록시간','배차시간','운행시간','잡힌시간','상차시간','하차시간']  # 시간 관련 특성 처리 방식에 따라 추가할 수 있음

# 범주형 특성에 대해 원-핫 인코딩 적용
edge_features, edge_encoder = apply_one_hot_encoding(edge_features, edge_categorical_columns)
edge_features_test, _ = apply_one_hot_encoding(edge_features_test, edge_categorical_columns)

# 수치형 특성에 대해 정규화 적용
edge_features, edge_scaler = apply_normalization(edge_features, edge_numeric_columns)
edge_features_test[edge_numeric_columns] = edge_scaler.transform(edge_features_test[edge_numeric_columns])


C:\Users\user\anaconda3\envs\ys\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\envs\ys\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# GNN 구현

In [18]:
# 차주키와 화물키의 유니크한 값들을 추출합니다.
unique_driver_ids = pd.unique(train_df['차주키'])
unique_cargo_ids = pd.unique(train_df['화물키'])

# 차주키와 화물키에 대해 고유한 인덱스를 부여합니다.
driver_index = {key: i for i, key in enumerate(unique_driver_ids)}
cargo_index = {key: i+len(unique_driver_ids) for i, key in enumerate(unique_cargo_ids)}

In [19]:
# 차주키와 화물키로부터 엣지 인덱스를 생성합니다.
# 차주키와 화물키를 노드 인덱스로 매핑하는 과정이 필요합니다.
driver_indices = [driver_index[key] for key in edge_features['차주키']]
cargo_indices = [cargo_index[key] for key in edge_features['화물키']]
edge_index = torch.tensor([driver_indices, cargo_indices], dtype=torch.long)


In [20]:
# 모든 화물에 대해 해당 화물을 운송한 차주의 인덱스를 찾습니다.
# 이때, 차주의 인덱스 매핑(dictionary)이 필요합니다.
driver_indices = [driver_index[key] for key in cargo_features['차주키']]

# 화물 인덱스는 화물 특성 데이터의 인덱스를 그대로 사용합니다.
cargo_indices = list(range(len(cargo_features)))

# 엣지 인덱스 생성
edge_index = torch.tensor([driver_indices, cargo_indices], dtype=torch.long)

# 화물 특성을 노드 특성으로 사용
node_features = torch.tensor(cargo_features.values, dtype=torch.float)

# 그래프 데이터 객체 생성
data = Data(x=node_features, edge_index=edge_index)


# TEST

In [21]:
# driver_features_test
# cargo_features_test
# edge_features_test

In [22]:
# 차주키와 화물키의 유니크한 값들을 추출합니다.
unique_driver_ids_test = pd.unique(test_df['차주키'])
unique_cargo_ids_test = pd.unique(test_df['화물키'])

# 차주키와 화물키에 대해 고유한 인덱스를 부여합니다.
driver_index_test = {key: i for i, key in enumerate(unique_driver_ids_test)}
cargo_index_test = {key: i+len(unique_driver_ids_test) for i, key in enumerate(unique_cargo_ids_test)}

In [23]:
# 차주키와 화물키로부터 엣지 인덱스를 생성합니다.
# 차주키와 화물키를 노드 인덱스로 매핑하는 과정이 필요합니다.
driver_indices_test = [driver_index_test[key] for key in edge_features_test['차주키']]
cargo_indices_test = [cargo_index_test[key] for key in edge_features_test['화물키']]
edge_index_test = torch.tensor([driver_indices_test, cargo_indices_test], dtype=torch.long)


In [24]:
# 모든 화물에 대해 해당 화물을 운송한 차주의 인덱스를 찾습니다.
# 이때, 차주의 인덱스 매핑(dictionary)이 필요합니다.
driver_indices_test = [driver_index_test[key] for key in cargo_features_test['차주키']]

# 화물 인덱스는 화물 특성 데이터의 인덱스를 그대로 사용합니다.
cargo_indices_test = list(range(len(cargo_features_test)))

# 엣지 인덱스 생성
edge_index_test = torch.tensor([driver_indices_test, cargo_indices_test], dtype=torch.long)

# 화물 특성을 노드 특성으로 사용
node_features_test = torch.tensor(cargo_features_test.values, dtype=torch.float)

# 그래프 데이터 객체 생성
test_data = Data(x=node_features_test, edge_index=edge_index_test)


In [25]:
data

Data(x=[758354, 51], edge_index=[2, 758354])

In [26]:
test_data

Data(x=[144844, 51], edge_index=[2, 144844])

In [27]:
# import matplotlib.pyplot as plt
# import networkx as nx
# from torch_geometric.utils import to_networkx

# # PyTorch Geometric Data 객체의 일부분으로부터 NetworkX 그래프를 생성합니다.
# # 여기서는 edge_index의 처음 10개 엣지만 사용합니다.
# sub_data = Data(x=data.x, edge_index=data.edge_index[:, :10])

# # NetworkX 그래프로 변환
# G_sub = to_networkx(sub_data, to_undirected=True)

# # 그래프 시각화
# plt.figure(figsize=(8, 8))
# nx.draw(G_sub, with_labels=True, node_color='skyblue', node_size=500, edge_color='k', linewidths=1, font_size=15)
# plt.show()


In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


Using device: cuda


# 초안 1

In [29]:
# GNN 모델 정의
class GNN(torch.nn.Module):
    def __init__(self, num_node_features, num_output_features):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 32)  # 입력 특성에서 32로 확장
        self.conv2 = GCNConv(32, 64)  # 32에서 64로 확장
        self.conv3 = GCNConv(64, 128)  # 64에서 128로 확장
        self.conv4 = GCNConv(128, 256)  # 128에서 256으로 확장
        self.decoder = torch.nn.Linear(256, num_output_features)  # 디코더는 256에서 원래 특성 크기로 축소

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.1, training=self.training)  # 드롭아웃 추가
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.1, training=self.training)  # 드롭아웃 추가
        x = F.relu(self.conv3(x, edge_index))
        x = F.dropout(x, p=0.1, training=self.training)  # 드롭아웃 추가
        x = F.relu(self.conv4(x, edge_index))
        
        # 마지막에 디코더를 통해 원래 차원으로 매핑
        x = self.decoder(x)

        return x

# 데이터와 모델을 CUDA로 옮깁니다 (GPU 사용 가능한 경우).
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

# num_node_features와 num_output_features 정의
num_node_features = data.num_node_features
num_output_features = data.num_node_features  # 원본 노드 특성의 차원 수와 동일

# 모델 인스턴스 생성 및 GPU로 옮깁니다.
model = GNN(num_node_features, num_output_features).to(device)

# 옵티마이저 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 학습률 조정

# 그래디언트 클리핑 (선택적)
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

# 모델 학습
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)  # 모델 출력
    loss = F.smooth_l1_loss(out, data.x)/500
    #loss = F.l1_loss(out, data.x)
    #loss = F.mse_loss(out, data.x)  # 재구성 오차 계산
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}: Loss: {loss.item()}')

Epoch 1: Loss: 3.3263392448425293
Epoch 2: Loss: 2.929079532623291
Epoch 3: Loss: 2.7526557445526123
Epoch 4: Loss: 2.658186674118042
Epoch 5: Loss: 2.61769700050354
Epoch 6: Loss: 2.588926076889038
Epoch 7: Loss: 2.5530476570129395
Epoch 8: Loss: 2.5170576572418213
Epoch 9: Loss: 2.4873783588409424
Epoch 10: Loss: 2.4624829292297363
Epoch 11: Loss: 2.440107822418213
Epoch 12: Loss: 2.4205234050750732
Epoch 13: Loss: 2.403373956680298
Epoch 14: Loss: 2.387141704559326
Epoch 15: Loss: 2.371040105819702
Epoch 16: Loss: 2.355769395828247
Epoch 17: Loss: 2.342987298965454
Epoch 18: Loss: 2.3312292098999023
Epoch 19: Loss: 2.3223609924316406
Epoch 20: Loss: 2.3138837814331055
Epoch 21: Loss: 2.304863691329956
Epoch 22: Loss: 2.2966599464416504
Epoch 23: Loss: 2.2891480922698975
Epoch 24: Loss: 2.2804183959960938
Epoch 25: Loss: 2.2714309692382812
Epoch 26: Loss: 2.26344895362854
Epoch 27: Loss: 2.255608081817627
Epoch 28: Loss: 2.2485640048980713
Epoch 29: Loss: 2.2415637969970703
Epoch 30:

In [30]:
data

Data(x=[758354, 51], edge_index=[2, 758354])

In [31]:
test_data

Data(x=[144844, 51], edge_index=[2, 144844])

In [32]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total number of parameters: {total_params}')

Total number of parameters: 58227


# 소수

In [65]:
# # 모델 및 테스트 데이터 준비
# model.eval()
# test_data = test_data.to(device)

# with torch.no_grad():
#     # 모델을 통해 전체 테스트 데이터에 대한 특성을 예측
#     predictions = model(test_data)

# # 화물 특성 벡터 준비 (여기서는 예시로 test_data.x를 사용)
# cargo_features = test_data.x

# # 각 차주별로 화물과의 코사인 유사도 계산 및 상위 N개 추천
# top_n = 30
# recommended_cargos_per_driver = {}

# # 최대 100명의 차주에 대해서만 추천 계산
# for driver_id in range(min(10, predictions.size(0))):
#     # 차주의 특성 벡터 추출
#     driver_features = predictions[driver_id].unsqueeze(0)  # (1, feature_size)
    
#     # 유사도를 저장할 리스트 초기화
#     similarity_scores = []
    
#     # 각 화물과의 코사인 유사도 계산
#     for cargo_feature in cargo_features:
#         similarity = F.cosine_similarity(driver_features, cargo_feature.unsqueeze(0))
#         similarity_scores.append(similarity.item())
    
#     # 상위 N개 화물의 인덱스 추출
#     top_cargos_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:top_n]
    
#     # 추천 화물 리스트 저장
#     recommended_cargos_per_driver[driver_id] = top_cargos_indices

# # 추천 결과 확인 (최대 5명의 차주에 대한 추천 결과만 출력)
# for driver_id, cargos in list(recommended_cargos_per_driver.items())[:5]:
#     # 인덱스를 다시 매핑하여 실제 값으로 변환
#     actual_cargos = [unique_cargo_ids[cargo_idx] for cargo_idx in cargos]
#     print(f"Driver {unique_driver_ids[driver_id]}: Recommended cargos: {actual_cargos}")


In [68]:
model.eval()
with torch.no_grad():
    # 모델을 통해 전체 테스트 데이터에 대한 노드 임베딩을 예측합니다.
    test_predictions = model(test_data)


# 차주 노드의 임베딩과 화물 노드의 임베딩을 분리합니다.
driver_embeddings_test = test_predictions[list(driver_index_test.values())]
cargo_embeddings_test = test_predictions[len(unique_driver_ids_test):]

# 각 차주별로 상위 N개의 화물을 추천합니다.
top_n = 30  # 상위 N개의 화물을 추천
recommended_cargos_per_driver_test = {}

for driver_id in unique_driver_ids_test:
    driver_idx = driver_index_test[driver_id]
    driver_embedding = driver_embeddings_test[driver_idx].unsqueeze(0)
    
    # 코사인 유사도를 계산하여 상위 N개의 화물을 선택합니다.
    similarity_scores = F.cosine_similarity(driver_embedding, cargo_embeddings_test)
    top_cargos_indices = similarity_scores.topk(top_n).indices
    top_cargos_ids = [unique_cargo_ids_test[idx] for idx in top_cargos_indices.tolist()]

    recommended_cargos_per_driver_test[driver_id] = top_cargos_ids

# 결과 출력
for driver_id, recommended_cargos in list(recommended_cargos_per_driver_test.items())[:5]:
    print(f"Driver {driver_id}: Recommended cargos: {recommended_cargos}")


Driver 56129: Recommended cargos: [118850, 121056, 124836, 118838, 118766, 125393, 118706, 120256, 118918, 119429, 118608, 119515, 118890, 118604, 119041, 123340, 118514, 126587, 133636, 118666, 118590, 102114, 125502, 118622, 118569, 126059, 108511350, 122253, 118562, 118541]
Driver 85477: Recommended cargos: [243372, 247219, 241699, 242427, 238708, 236953, 218244, 235373, 208478, 209347, 205557, 206893, 218116, 215377, 203503, 202228, 144583, 146136, 135963, 137239, 120934, 120256, 122926, 129297, 197229, 177566, 171601, 173149, 197239, 195389]
Driver 22569: Recommended cargos: [247219, 247399, 242427, 243372, 241699, 238708, 235373, 236953, 215377, 208478, 205557, 209347, 218244, 218116, 203503, 202228, 144583, 146136, 135963, 137239, 120934, 120256, 122926, 129297, 197229, 177566, 171601, 173149, 197239, 195389]
Driver 84744: Recommended cargos: [243372, 247219, 241699, 242427, 238708, 236953, 218244, 235373, 208478, 209347, 205557, 206893, 218116, 215377, 203503, 202228, 144583, 1

In [105]:
# 결과를 저장할 빈 리스트 초기화
flattened_recommendations = []

# 추천된 화물 리스트를 차주별로 펼치기
for driver_id, recommended_cargos in recommended_cargos_per_driver_test.items():
    for cargo_id in recommended_cargos:
        flattened_recommendations.append({'Driver': driver_id, 'Cargo': cargo_id})

# 결과를 DataFrame으로 변환
flattened_df = pd.DataFrame(flattened_recommendations)

# 결과 확인
print(flattened_df)


        Driver   Cargo
0        56129  118850
1        56129  121056
2        56129  124836
3        56129  118838
4        56129  118766
...        ...     ...
459955   89302  177566
459956   89302  171601
459957   89302  173149
459958   89302  197239
459959   89302  195389

[459960 rows x 2 columns]


In [107]:
#flattened_df.to_csv("flattened_df.csv", index = False)

In [85]:
actual_cargos=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/actual_cargos_per_driver.csv',encoding='cp949')
actual_cargos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15332 entries, 0 to 15331
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Driver         15332 non-null  int64 
 1   Actual cargos  15332 non-null  object
dtypes: int64(1), object(1)
memory usage: 239.7+ KB


In [86]:
actual_cargos

,Driver,Actual cargos
0,4,"[119099, 957508, 957471]"
1,6,"[970366, 1028529]"
2,12,[141758]
3,22,"[139338, 146412, 561963, 562222, 760384, 1018933]"
4,45,[520151]
...,...,...
15327,99995,"[702904, 833650, 968319]"
15328,99998,"[169910, 213450, 471035, 494979, 518844, 54558..."
15329,99999,"[171149, 636294]"
15330,33431833,[825527]


In [91]:
# `Actual cargos` 컬럼을 explode하여 각 화물 ID를 별도의 행으로 확장
actual_cargos_exploded = actual_cargos.explode('Actual cargos')

# 컬럼명 변경: 'Actual cargos'를 'Actual cargo'로
actual_cargos_exploded.rename(columns={'Actual cargos': 'Actual cargo'}, inplace=True)

# 결과 확인
print(actual_cargos_exploded.head())

   Driver                                       Actual cargo
0       4                           [119099, 957508, 957471]
1       6                                  [970366, 1028529]
2      12                                           [141758]
3      22  [139338, 146412, 561963, 562222, 760384, 1018933]
4      45                                           [520151]


In [101]:
# 결과를 저장할 빈 리스트 초기화
flattened_cargo = []

# 실제 화물 데이터를 차주별로 펼치기
for index, row in actual_cargos_exploded.iterrows():
    flattened_cargo.append({'Driver': row['Driver'], 'Cargo': row['Actual cargo']})

# 결과를 DataFrame으로 변환
flattened_df = pd.DataFrame(flattened_cargo)

# 결과 확인
print(flattened_df.head())


   Driver                                              Cargo
0       4                           [119099, 957508, 957471]
1       6                                  [970366, 1028529]
2      12                                           [141758]
3      22  [139338, 146412, 561963, 562222, 760384, 1018933]
4      45                                           [520151]


In [102]:
# 제공된 데이터를 바탕으로 각 차주(Driver)와 화물(Cargo)을 펼친 후, 각각을 별도의 행으로 나타내기 위한 작업
for index, row in exploded_df.iterrows():
    print(row['Driver'], row['Actual cargo'])

4 [119099, 957508, 957471]
6 [970366, 1028529]
12 [141758]
22 [139338, 146412, 561963, 562222, 760384, 1018933]
45 [520151]
50 [147188, 165639, 198494, 310268, 329282, 683458, 686785, 761431, 742028, 786580, 786598, 787098, 834087, 902060, 902117, 957300, 1014673, 1018453, 1018709, 1018969, 1210678]
55 [903915]
70 [338115]
79 [311378, 649541]
486 [147531, 147859, 295802, 295790, 295795, 306389, 384806, 519972, 520009, 520061, 657924, 694891, 712140, 789442, 789551, 898628, 954782, 954810, 1030264]
1002 [168584, 208699, 514121, 537620, 787899, 890780, 975323, 1022295, 1174099, 1177547, 1213709]
1008 [135101, 238420, 249732, 260523, 336295, 341495, 545131, 658716, 690822, 904889, 973910, 1024558, 1035227, 1169878]
1009 [170912, 192609, 213507, 348687, 530291, 553600, 560753, 739254, 751856, 1018708, 1033488, 1130278, 1212727]
1012 [127685, 336484, 345953, 392344, 479186, 553496, 651332, 690834, 699597, 744093, 821074, 889919, 965266, 978468, 1187770]
1019 [245878, 386979, 703318, 828309]

In [99]:
# 각 Driver 별로 추천한 Cargo와 실제 Cargo를 비교하여 Precision, Recall, F1 계산
results = []
for driver in set(flattened_df['Driver']).union(set(exploded_df['Driver'])):
    recommended_cargos = set(flattened_df[flattened_df['Driver'] == driver]['Cargo'])
    actual_cargos = set(exploded_df[exploded_df['Driver'] == driver]['Actual cargo'])
    
    # 교집합과 각 집합의 크기 계산
    intersection = recommended_cargos.intersection(actual_cargos)
    precision = len(intersection) / len(recommended_cargos) if recommended_cargos else 0
    recall = len(intersection) / len(actual_cargos) if actual_cargos else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0
    
    results.append({
        'Driver': driver,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

# 결과를 DataFrame으로 변환하여 확인
results_df = pd.DataFrame(results)
results_df.head()  # 상위 결과 출력


,Driver,Precision,Recall,F1 Score
0,4,1.0,1.0,1.0
1,6,1.0,1.0,1.0
2,12,1.0,1.0,1.0
3,22,1.0,1.0,1.0
4,45,1.0,1.0,1.0


In [100]:
# 각 Driver 별로 추천한 Cargo와 실제 Cargo를 비교하여 Precision, Recall, F1 계산
results = []
for driver in set(flattened_df['Driver']).union(set(exploded_df['Driver'])):
    recommended_cargos = set(flattened_df[flattened_df['Driver'] == driver]['Cargo'])
    actual_cargos = set(exploded_df[exploded_df['Driver'] == driver]['Actual cargo'])
    
    # 교집합과 각 집합의 크기 계산
    intersection = recommended_cargos.intersection(actual_cargos)
    precision = len(intersection) / len(recommended_cargos) if recommended_cargos else 0
    recall = len(intersection) / len(actual_cargos) if actual_cargos else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0
    
    results.append({
        'Driver': driver,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

# 결과를 DataFrame으로 변환하여 확인
results_df = pd.DataFrame(results)
results_df.head()  # 상위 결과 출력


,Driver,Precision,Recall,F1 Score
0,4,1.0,1.0,1.0
1,6,1.0,1.0,1.0
2,12,1.0,1.0,1.0
3,22,1.0,1.0,1.0
4,45,1.0,1.0,1.0


# 전체

In [40]:
# K값 설정
K_values = [10, 20, 30]

# 성능 평가 결과를 저장할 딕셔너리
performance_metrics = {k: {'recall': [], 'precision': []} for k in K_values}

for driver_id, recommended_cargos in recommended_cargos_per_driver.items():
    actual_cargos = actual_cargos_per_driver.get(driver_id)
    if actual_cargos is None:
        continue

    actual_set = set(actual_cargos)

    for k in K_values:
        recommended_k = set(recommended_cargos[:k])
        
        # Recall@K: 실제 운송된 화물 중 추천된 화물의 비율
        recall_at_k = len(recommended_k.intersection(actual_set)) / len(actual_set)
        
        # Precision@K: 추천된 화물 중 실제로 운송된 화물의 비율
        precision_at_k = len(recommended_k.intersection(actual_set)) / k
        
        performance_metrics[k]['recall'].append(recall_at_k)
        performance_metrics[k]['precision'].append(precision_at_k)

# 평균 성능 평가 결과 출력
for k in K_values:
    avg_recall = sum(performance_metrics[k]['recall']) / len(performance_metrics[k]['recall'])
    avg_precision = sum(performance_metrics[k]['precision']) / len(performance_metrics[k]['precision'])
    
    print(f"Average Recall@{k}: {avg_recall:.4f}")
    print(f"Average Precision@{k}: {avg_precision:.4f}")


NameError: name 'actual_cargos_per_driver' is not defined

In [ ]:
# 모델 및 테스트 데이터 준비
model.eval()
test_data = test_data.to(device)

with torch.no_grad():
    # 모델을 통해 전체 테스트 데이터에 대한 특성을 예측
    predictions = model(test_data)

# 화물 특성 벡터 준비 (여기서는 예시로 test_data.x를 사용)
cargo_features = test_data.x

# 각 차주별로 화물과의 코사인 유사도 계산 및 상위 N개 추천
top_n = 20
recommended_cargos_per_driver = {}

for driver_id in range(predictions.size(0)):
    # 차주의 특성 벡터 추출
    driver_features = predictions[driver_id].unsqueeze(0)  # (1, feature_size)
    
    # 유사도를 저장할 리스트 초기화
    similarity_scores = []
    
    # 각 화물과의 코사인 유사도 계산
    for cargo_feature in cargo_features:
        similarity = F.cosine_similarity(driver_features, cargo_feature.unsqueeze(0))
        similarity_scores.append(similarity.item())
    
    # 상위 N개 화물의 인덱스 추출
    top_cargos_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:top_n]
    
    # 추천 화물 리스트 저장
    recommended_cargos_per_driver[driver_id] = top_cargos_indices

# 추천 결과 확인 (처음 1명의 차주에 대한 추천 결과만 출력)
for driver_id, cargos in list(recommended_cargos_per_driver.items())[:10]:
    # 인덱스를 다시 매핑하여 실제 값으로 변환
    actual_cargos = [unique_cargo_ids[cargo_idx] for cargo_idx in cargos]
    print(f"Driver {unique_driver_ids[driver_id]}: Recommended cargos: {actual_cargos}")

In [36]:
import pandas as pd

# 예시 데이터 준비
recommended_cargos_per_driver = {
    47982: [92238518, 107741141, 92244139, 92244177, 92246462, 92250172, 107765500, 107780487, 107786738, 107787105,
            107788122, 107788117, 92302076, 92302888, 92304846, 107828124, 92317388, 92218976, 92218977, 107700731],
    85445: [92238518, 107741141, 92244139, 92244177, 92246462, 92250172, 107765500, 107780487, 107786738, 107787105,
            107788122, 107788117, 92302076, 92302888, 92304846, 107828124, 92317388, 92218976, 92218977, 107700731],
    # 추가 차주 데이터...
}

actual_cargos_per_driver = {
    47982: [92244139, 107788122, 107765500],
    85445: [92238518, 92304846],
    # 추가 차주 데이터...
}

# K값 설정
K_values = [10, 20, 30]

# 성능 평가 결과를 저장할 딕셔너리
performance_metrics = {k: {'recall': [], 'precision': []} for k in K_values}

for driver_id, recommended_cargos in recommended_cargos_per_driver.items():
    actual_cargos = actual_cargos_per_driver.get(driver_id)
    if actual_cargos is None:
        continue

    actual_set = set(actual_cargos)

    for k in K_values:
        recommended_k = set(recommended_cargos[:k])
        
        # Recall@K: 실제 운송된 화물 중 추천된 화물의 비율
        recall_at_k = len(recommended_k.intersection(actual_set)) / len(actual_set) if actual_set else 0
        
        # Precision@K: 추천된 화물 중 실제로 운송된 화물의 비율
        precision_at_k = len(recommended_k.intersection(actual_set)) / k if k else 0
        
        performance_metrics[k]['recall'].append(recall_at_k)
        performance_metrics[k]['precision'].append(precision_at_k)

# 평균 성능 평가 결과 출력
for k in K_values:
    avg_recall = sum(performance_metrics[k]['recall']) / len(performance_metrics[k]['recall']) if performance_metrics[k]['recall'] else 0
    avg_precision = sum(performance_metrics[k]['precision']) / len(performance_metrics[k]['precision']) if performance_metrics[k]['precision'] else 0
    
    print(f"Average Recall@{k}: {avg_recall:.4f}")
    print(f"Average Precision@{k}: {avg_precision:.4f}")


Average Recall@10: 0.5833
Average Precision@10: 0.1500
Average Recall@20: 1.0000
Average Precision@20: 0.1250
Average Recall@30: 1.0000
Average Precision@30: 0.0833


# 초안 2 -> 다시 그래프 만들기 ! 

In [ ]:
driver_features
driver_features_test

cargo_features
cargo_features_test

edge_features
edge_features_test

In [ ]:
# import torch
# from torch_geometric.data import Data

# # 예시: 노드 특성과 엣지 인덱스 생성 (실제 데이터에 맞게 조정 필요)
# driver_features_tensor = torch.tensor(driver_features.values, dtype=torch.float)
# cargo_features_tensor = torch.tensor(cargo_features.values, dtype=torch.float)
# node_features = torch.cat((driver_features_tensor, cargo_features_tensor), dim=0)

# # 엣지 인덱스 생성
# driver_indices = [driver_index[key] for key in edge_features['차주키']]
# cargo_indices = [cargo_index[key] for key in edge_features['화물키']]
# edge_index = torch.tensor([driver_indices, cargo_indices], dtype=torch.long)

# # 그래프 데이터 객체 생성
# graph_data = Data(x=node_features, edge_index=edge_index)


In [ ]:
import networkx as nx

# 빈 그래프 생성
G = nx.Graph()

# 차주 노드 추가
for i in range(len(driver_features)):
    G.add_node(f"driver_{i}", features=driver_features[i])

# 화물 노드 추가
for i in range(len(cargo_features)):
    G.add_node(f"cargo_{i}", features=cargo_features[i])

# 그래프 정보 출력
print(nx.info(G))

In [ ]:
# 차원 수를 맞추기 위한 더미 특성 추가
# 예: driver_features_tensor에 더미 특성 추가
num_features = max(driver_features_tensor.shape[1], cargo_features_tensor.shape[1])
driver_features_padded = torch.cat([driver_features_tensor, torch.zeros(driver_features_tensor.shape[0], num_features - driver_features_tensor.shape[1])], dim=1)
cargo_features_padded = torch.cat([cargo_features_tensor, torch.zeros(cargo_features_tensor.shape[0], num_features - cargo_features_tensor.shape[1])], dim=1)

# 이제 두 텐서를 결합할 수 있음
node_features = torch.cat((driver_features_padded, cargo_features_padded), dim=0)

In [ ]:
edge_index = torch.tensor([driver_indices, cargo_indices], dtype=torch.long)

# 그래프 데이터 객체 생성
data = Data(x=node_features, edge_index=edge_index)

In [ ]:
data

In [ ]:
# 모델 디바이스 확인
model_device = next(model.parameters()).device
print(f'Model is on {model_device}')

# 데이터 디바이스 확인 (test_data의 예시)
data_device = test_data.x.device
print(f'Data is on {data_device}')

# 두 디바이스가 일치하는지 확인
assert model_device == data_device, "Model and data are on different devices!"


In [ ]:
print("Embeddings size:", embeddings.size())
print("Max driver index:", max(driver_indices))
print("Max cargo index:", max(cargo_indices))
print("Min driver index:", min(driver_indices))
print("Min cargo index:", min(cargo_indices))

assert max(driver_indices) < embeddings.size(0), "Driver index out of range"
assert max(cargo_indices) < embeddings.size(0), "Cargo index out of range"
assert min(driver_indices) >= 0, "Driver index out of range"
assert min(cargo_indices) >= 0, "Cargo index out of range"
